In [3]:
import seaborn as sns
import importlib
import paper_gen as pg
importlib.reload(pg)
sns.set(context="paper", style="white", font_scale=1)

path = '/tmp/table1'
ed_th = [.1, .2, .3, .5]
flags, dists, stats = pg.get_summary(path=path, thresh=ed_th)
pg.texify_table(flags=flags, stats=stats, thresh=ed_th)
pg.gen_fig_s1(flags=flags, dists=dists)
pg.gen_fig_s2(flags=flags, dists=dists, ed_th=ed_th)

path = '/tmp/fig1/'
pg.gen_fig1(fig1_path=path)

path = '/tmp/fig2'
pg.gen_fig2(path=path)
